In [ ]:
# GPU: 32*40 in ?.??s = ???/s
# CPU: 32*8 in ??.?s = ?/s

In [ ]:
#Pkg.checkout("Knet","ilkarman") # make sure we have the right Knet version
#Pkg.build("Knet")
using Knet
include(Knet.dir("examples","resnet", "resnetlib.jl"))
using ResNetLib: resnet50init, resnet50
println("OS: ", Sys.KERNEL)
println("Julia: ", VERSION)
println("Knet: ", Pkg.installed("Knet"))

In [ ]:
;cat /proc/cpuinfo '|' grep processor '|' wc -l

In [ ]:
;nvidia-smi --query-gpu=gpu_name --format=csv

In [ ]:
const BATCH_SIZE = 32
const RESNET_FEATURES = 2048
const BATCHES_GPU = 40
const BATCHES_CPU = 8

In [ ]:
# Create batches of fake data
function fakedata(batches; atype=KnetArray)
    x = rand(Float32, 224, 224, 3, BATCH_SIZE * batches)
    minibatch(x, BATCH_SIZE, xtype=atype)
end

In [ ]:
# Return features from classifier
function predictfn(weights, moments, data)
    out = Any[]
    for x in data
        pred = resnet50(weights, moments, x; stage=5)
        push!(out, mat(pred))
    end
    return hcat(out...)
end

## 1. GPU

In [ ]:
# Initialize resnet weights and fake data
gpuweights, gpumoments = resnet50init(;stage=5, trained=true, atype=KnetArray)
gpudata = fakedata(BATCHES_GPU, atype=KnetArray);

In [ ]:
info("Cold start")
@time predictfn(gpuweights, gpumoments, gpudata);

In [ ]:
info("Benchmarking")
@time predictfn(gpuweights, gpumoments, gpudata);

## 2. CPU

In [ ]:
# Initialize resnet weights and data
cpuweights, cpumoments = resnet50init(;stage=5, trained=true, atype=Array)
cpudata = fakedata(BATCHES_CPU, atype=Array);

In [ ]:
info("Cold start")
@time predictfn(cpuweights, cpumoments, cpudata);

In [ ]:
info("Benchmarking")
@time predictfn(cpuweights, cpumoments, cpudata);